<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
## get the data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-05-25 07:14:18--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  13.7MB/s    in 19s     

2020-05-25 07:14:38 (13.0 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [2]:
!unzip -qq concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [1]:
# 1. Import libraries, modules, and packages you will need. Make sure to import the preprocess_input function from keras.applications.vgg16.
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import VGG16
from keras.applications.resnet50 import preprocess_input

Using TensorFlow backend.


In [2]:
# 2. Use a batch size of 100 images for both training and validation.

num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [3]:
# 3. Construct an ImageDataGenerator for the training set and another one for the validation set. 
#    VGG16 was originally trained on 224 × 224 images, 
#    so make sure to address that when defining the ImageDataGenerator instances.

data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')


Found 30001 images belonging to 2 classes.


In [4]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [5]:
# 4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
model = Sequential()
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

In [6]:
model.add(Dense(num_classes, activation='softmax'))
model.layers
model.layers[0].layers
model.layers[0].trainable = False
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [7]:
# 5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# 6. Fit the model on the augmented data using the ImageDataGenerators.
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [9]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
301/301 [==============================] - 10236s 34s/step - loss: 0.1095 - accuracy: 0.9626 - val_loss: 0.0036 - val_accuracy: 0.9953
Epoch 2/2
301/301 [==============================] - 8814s 29s/step - loss: 0.0196 - accuracy: 0.9955 - val_loss: 0.0010 - val_accuracy: 0.9966


In [10]:
model.save('classifier_vgg16_model.h5')

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [13]:
# 1. Load your saved model that was built using the ResNet50 model.
from keras.models import load_model
model_ResNet50 = load_model("classifier_resnet_model.h5")
model_VGG16 = load_model("classifier_vgg16_model.h5")

In [14]:
# 2. Construct an ImageDataGenerator for the test set. 
#    For this ImageDataGenerator instance, you only need to pass the directory of 
#      the test images, target size, and the shuffle parameter and set it to False.

test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle=False)

Found 500 images belonging to 2 classes.


In [15]:
# 3. Use the evaluate_generator method to evaluate your models on the test data, 
#    by passing the above ImageDataGenerator as an argument. 

res_ResNet50 =  model_ResNet50.evaluate_generator(test_generator)
res_ResNet50

[0.1493256390094757, 0.9259999990463257]

In [16]:
res_VGG16 =  model_VGG16.evaluate_generator(test_generator)
res_VGG16

[0.002397971460595727, 0.9959999918937683]

In [23]:
# Print the performance of the classifier using the VGG16 pre-trained model.
# Print the performance of the classifier using the ResNet pre-trained model.

print("=" * 80)
print("the performance of the classifier using the VGG16 ResNet pre-trained model")
print(f"data loss : {res_VGG16[0]}, accuracy : {res_VGG16[1]}")
print("=" * 80)
print("the performance of the classifier using the ResNet pre-trained model")
print(f"data loss : {res_ResNet50[0]}, accuracy : {res_ResNet50[1]}")
print("=" * 80)

the performance of the classifier using the VGG16 ResNet pre-trained model
data loss : 0.002397971460595727, accuracy : 0.9959999918937683
the performance of the classifier using the ResNet pre-trained model
data loss : 0.1493256390094757, accuracy : 0.9259999990463257


## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [21]:
# 1. Use the predict_generator method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the predict_generator method here.
predict_ResNet50 =  model_ResNet50.predict_generator(test_generator)

In [22]:
predict_VGG16 =  model_VGG16.predict_generator(test_generator)

In [27]:
# 2. Report the class predictions of the first five images in the test set. You should print something list this:
print("=" * 80)
print("the first 5 predictions of the classifier using the VGG16 pre-trained model")
print( predict_VGG16[0:5] )
for i in range(5):
    temp = "Positive" if predict_VGG16[i][0] > predict_VGG16[i][1] else "Negative"
    print(temp)
print("=" * 80)
print("the first five predictions of the classifier using the ResNet pre-trained model")
print( predict_ResNet50[0:5] )
for i in range(5):
    temp = "Positive" if predict_ResNet50[i][0] > predict_ResNet50[i][1] else "Negative"
    print(temp)
print("=" * 80)

the first 5 predictions of the classifier using the VGG16 pre-trained model
[[0.99141055 0.00858948]
 [0.9382563  0.06174369]
 [0.9687282  0.03127184]
 [0.98407656 0.01592343]
 [0.9759386  0.02406139]]
Positive
Positive
Positive
Positive
Positive
the first five predictions of the classifier using the ResNet pre-trained model
[[9.9997115e-01 2.8865486e-05]
 [9.9997032e-01 2.9712841e-05]
 [9.9996698e-01 3.2993277e-05]
 [9.9989748e-01 1.0254774e-04]
 [9.9932659e-01 6.7346200e-04]]
Positive
Positive
Positive
Positive
Positive


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).